# Non-Parametric, Unsupervised UMAP and HDBSCAN

In [198]:
#Imports

### Unfortunately, UMAP takes a while to import. One of its dependencies (pynndescent) uses numba, 
### which is the cause of the performance bottleneck here.

import cv2
import hdbscan
import json 
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import re
import seaborn as sns
import skimage 
import umap
import umap.plot

from bokeh.embed import json_item
from bokeh.plotting import show as bokeh_show, output_notebook
from glob import glob
from hdbscan import HDBSCAN
from IPython.display import display, HTML, Image, Javascript
from pathlib import Path
from sklearn.preprocessing import StandardScaler

In [97]:
#Constants
IMG_SIZE = 256
IMAGES_DIR = '../images/final_to_match'
MAX_FILES = 300

We first load the images...

In [98]:
files = Path(IMAGES_DIR) 
image_files = []

for file in files.rglob('*'):
    ext = file.suffix.lower()
    if ext == ".jpg": image_files.append(file)
    if len(image_files) >= MAX_FILES: break
    
image_files[0:4]

[PosixPath('../images/final_to_match/HG_170305_0027_DE_P4.jpg'),
 PosixPath('../images/final_to_match/HG_170305_0041_GF_P5.JPG'),
 PosixPath('../images/final_to_match/HG_170305_0046_GF_P6.JPG'),
 PosixPath('../images/final_to_match/HG_170305_0051_GF_P1.JPG')]

Then apply additional preprocessing...

In [163]:
images = []
names = []
files = []

for i, file in enumerate(image_files):
    file_str = str(file)
    image = cv2.imread(file_str)
    names.append(file.stem)
    files.append(file_str)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image.astype('float32')
    #image = (image - image.mean(axis=(0, 1), keepdims = True)) / image.std(axis=(0, 1), keepdims = True)
    #image = cv2.normalize(image, None, 0, 1, cv2.NORM_MINMAX)
    images.append(image)
    
    progress = i/MAX_FILES * 100
    if progress % 5 == 0: print(f'{progress}% done')
        
print("Complete!")

0.0% done
5.0% done
10.0% done
15.0% done
20.0% done
25.0% done
30.0% done
35.0% done
40.0% done
45.0% done
50.0% done
60.0% done
65.0% done
70.0% done
75.0% done
80.0% done
85.0% done
90.0% done
95.0% done
Complete!


We then convert it to a numpy array.

In [199]:
images = np.asarray(images)
images.shape

(300, 256, 256)

In [223]:
data = images.reshape((images.shape[0], -1))
#data = StandardScaler().fit_transform(data)
data.shape

(300, 65536)

We then fit UMAP to the data.

In [224]:
mapper = umap.UMAP(
    n_neighbors = 200,
    min_dist = 0.5,
    n_components = 2,
    metric = 'euclidean',
    random_state = 100,
).fit(data)

umap_res = mapper.transform(data)

We apply HDBSCAN to the UMAP results.

In [225]:
cluster = HDBSCAN(
    algorithm ='best', 
    approx_min_span_tree = True,
    gen_min_span_tree = False, 
    leaf_size = 40, 
    metric='euclidean', 
    min_cluster_size = 15,
    min_samples = 15, 
    p = None
).fit(umap_res)

In [226]:
hover_data = pd.DataFrame(
    index = np.arange(
        data.shape[0] 
    ), 
    data = {
        'index' : np.arange(data.shape[0]),
        'name': names,
        "file": files,
        'class': cluster.labels_,
        'probability': cluster.probabilities_,
        'outlier': [ 1 if item == -1 else 0 for item in cluster.labels_ ]
    }
)

In [227]:
hover_data.head(10).drop("index", axis = 1)

,name,file,class,probability,outlier
0,HG_170305_0027_DE_P4,../images/final_to_match/HG_170305_0027_DE_P4.jpg,1,1.000000,0
1,HG_170305_0041_GF_P5,../images/final_to_match/HG_170305_0041_GF_P5.JPG,1,0.995755,0
2,HG_170305_0046_GF_P6,../images/final_to_match/HG_170305_0046_GF_P6.JPG,-1,0.000000,1
3,HG_170305_0051_GF_P1,../images/final_to_match/HG_170305_0051_GF_P1.JPG,1,0.983698,0
4,HG_170305_0067_GF_P7,../images/final_to_match/HG_170305_0067_GF_P7.JPG,-1,0.000000,1
5,HG_170305_0073_GF_N3,../images/final_to_match/HG_170305_0073_GF_N3.JPG,-1,0.000000,1
6,HG_170305_0091_GF_P8,../images/final_to_match/HG_170305_0091_GF_P8.JPG,-1,0.000000,1
7,HG_170305_0110_GF_N6,../images/final_to_match/HG_170305_0110_GF_N6.JPG,-1,0.000000,1
8,HG_170305_0115_GF_N5,../images/final_to_match/HG_170305_0115_GF_N5.JPG,-1,0.000000,1
9,HG_170305_0140_GF_N4,../images/final_to_match/HG_170305_0140_GF_N4.JPG,-1,0.000000,1


In [228]:
p = umap.plot.interactive(mapper, labels = cluster.labels_,hover_data = hover_data, point_size = 5)
output_notebook()
bokeh_show(p)

Loading BokehJS ...

In [221]:
p_json = json.dumps(json_item(p))
p_json[0:150]

'{"target_id": null, "root_id": "5898", "doc": {"defs": [], "roots": {"references": [{"attributes": {"background_fill_color": "white", "below": [{"id":'

In [222]:
display(HTML('<div id="umap"></div>'))
Javascript(f'''Bokeh.embed.embed_item({p_json}, "umap")''')

<IPython.core.display.Javascript object>